In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
%run PARAMETERS.py
NAME = 'APG'
PATH = NAME + '-bmnist-3digits-galileo-96'
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

from model_operations import Init_models_test, Save_models

models, AT = Init_models_test(K=K,
                                D=D,
                                FRAME_PIXELS=FRAME_PIXELS, 
                                DIGIT_PIXELS=DIGIT_PIXELS, 
                                HIDDEN_LIST=HIDDEN_LIST, 
                                Sigma0=Sigma0,
                                CUDA=CUDA, 
                                DEVICE=DEVICE, 
                                lr=LEARNING_RATE, 
                                RESTORE=True, 
                                PATH=PATH)

mnist_mean = mnist_mean.cuda().to(DEVICE)

In [3]:
K=3
NUM_GROUPS_TEST = 1
T_test =20
data_path_test = "/data/hao/apg_data/mnist/test_T%dD%d/" % (T_test, K)


In [4]:
from apg_galileo import APG
from viz_mc import Viz_MC
from testing import Eval
BATCH_SIZE = 5
Viz_INTERVAL = 5
FS = 30*1.5
TITLE_FS = 20
LW= 3

COLORS = ['#AA3377', '#EE7733', '#009988', '#0077BB', '#BBBBBB', '#EE3377']
apg = APG(models=models, 
          frame_size=FRAME_PIXELS,
          AT=AT, 
          K=K, 
          D=D, 
          T=T_test, 
          mnist_mean=mnist_mean, 
          training=False)

evaluator = Eval(DATA_PATH=data_path_test,
                 NUM_GROUPS=NUM_GROUPS_TEST, 
                 K=K, 
                 D=D)

# visualizer = Viz_MC(K=K, 
#                     viz_interval=Viz_INTERVAL, 
#                     FS=FS, 
#                     TITLE_FS=TITLE_FS, 
#                     COLORS=COLORS,
#                     LW=LW)


In [5]:
def Test_uniform(APG, group_ptr, data_ptr, APG_STEPS, S, B, DEVICE, PATH):
    """
    training function
    """

    group_ind = torch.arange(NUM_GROUPS_TEST)
    data_g = torch.from_numpy(np.load(data_path_test + 'ob_%d.npy' % group_ind[group_ptr])).float()
    NUM_DATA= data_g.shape[0]
    indices = torch.arange(NUM_DATA)
    MSEs = []
    LJs = []
    NUM_BATCHES = int(NUM_DATA / B)
    for b in range(100, NUM_BATCHES, 1):
        time_start = time.time()
        frames = data_g[data_ptr*B : (data_ptr+1)*B]
        frames = frames.repeat(S, 1, 1, 1, 1).cuda().to(DEVICE)
        metrics = APG.Sweeps(APG_STEPS, S, B, frames)
        MSE = ((metrics['recon'].cpu() - frames.cpu())**2).sum(-1).sum(-1).mean(-1).mean(-1).mean(-1)
        MSEs.append(MSE.unsqueeze(0))
        time_end = time.time()
        print('b=%d, (%ds)' % (b, time_end - time_start))
    MSEs = torch.cat(MSEs, 0)
    return MSEs

In [6]:
# DATA_PTR = 1
GROUP_PTR = 2
DATA_PTR= 4

APG_STEPS = 10

MSEs = Test_uniform(APG=apg, 
                        group_ptr=0,
                        data_ptr=DATA_PTR,
                        APG_STEPS=APG_STEPS, 
                        S=50,
                        B=BATCH_SIZE,
                        DEVICE=DEVICE,
                        PATH=PATH)

np.save('mse-p2-T=%d-D=%d' % (T_test, K), MSEs.data.numpy())

b=100, (15s)
b=101, (14s)
b=102, (14s)
b=103, (15s)
b=104, (15s)
b=105, (17s)
b=106, (17s)
b=107, (15s)
b=108, (14s)
b=109, (15s)
b=110, (14s)
b=111, (15s)
b=112, (16s)
b=113, (17s)
b=114, (16s)
b=115, (15s)
b=116, (14s)
b=117, (14s)
b=118, (14s)
b=119, (16s)
b=120, (17s)
b=121, (17s)
b=122, (15s)
b=123, (14s)
b=124, (14s)
b=125, (14s)
b=126, (15s)
b=127, (16s)
b=128, (17s)
b=129, (16s)
b=130, (15s)
b=131, (14s)
b=132, (14s)
b=133, (14s)
b=134, (15s)
b=135, (17s)
b=136, (17s)
b=137, (15s)
b=138, (14s)
b=139, (14s)
b=140, (14s)
b=141, (15s)
b=142, (16s)
b=143, (17s)
b=144, (16s)
b=145, (15s)
b=146, (14s)
b=147, (14s)
b=148, (14s)
b=149, (15s)
b=150, (16s)
b=151, (17s)
b=152, (16s)
b=153, (14s)
b=154, (14s)
b=155, (14s)
b=156, (15s)
b=157, (16s)
b=158, (17s)
b=159, (16s)
b=160, (15s)
b=161, (14s)
b=162, (14s)
b=163, (14s)
b=164, (15s)
b=165, (16s)
b=166, (17s)
b=167, (16s)
b=168, (14s)
b=169, (14s)
b=170, (14s)
b=171, (15s)
b=172, (16s)
b=173, (17s)
b=174, (16s)
b=175, (15s)
b=176, (14s)

In [7]:
np.save('mse-T=%d-D=%d' % (T_test, K), MSEs.data.numpy())